In [39]:
# # Uncomment this for using at Google Colab
# !pip -q install nltk
# !pip -q install torch
# !pip -q install gensim
# !pip -q install wordcloud
# !pip -q install torchvision

# from google.colab import drive
# drive.mount('/content/gdrive')
# path = "gdrive/My Drive/AML/Project/Dataset/"



import os
import csv
import nltk
import pickle
import string
import gensim
import numpy as np
import pandas as pd
from sklearn.svm import SVC
from tqdm import tqdm_notebook
import matplotlib.pyplot as plt
from collections import Counter
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize
from sklearn.neural_network import MLPClassifier
from sklearn.feature_extraction.text import TfidfVectorizer




import torch
import torchvision
import seaborn as sb
from torch import nn
from tqdm import tqdm_notebook
import matplotlib.pyplot as plt
import torch.nn.functional as F
from torchvision import transforms
from torch.autograd import Variable
from torch.utils.data import DataLoader, Dataset

In [40]:
path = '/home/arpan_aml/Project/dataset/'
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
n_epochs = 100

In [41]:
with open(path+'dataset.pkl', 'rb') as fp:
    data = pickle.load(fp)
trainX = data['trainX']
trainY = data['trainY']
testX = data['testX']
testY = data['testY']

In [42]:
def get_string(x):
    ans = ''
    for i in x:
        ans += (i+' ')
    return ans

### TfIdf Embedding

We tried creating a tf-idf embedding by considering all the questions as a single corporus. But dimension(no of unique words) was 84334. So for each question it had hugely sparsed embedding vector and no system was able to handle this kind of large dimentional input preprocessing, so we couldn't even reduce the dimension using some kind of dimentionality reduction technique(e.g.-PCA).

In [6]:
data = []
for i in range(len(trainX)):
    data.append(get_string(trainX[i][0]))
    data.append(get_string(trainX[i][1]))

In [7]:
vectorizer = TfidfVectorizer()
vectorizer.fit(data)

TfidfVectorizer(analyzer='word', binary=False, decode_error='strict',
        dtype=<class 'numpy.float64'>, encoding='utf-8', input='content',
        lowercase=True, max_df=1.0, max_features=None, min_df=1,
        ngram_range=(1, 1), norm='l2', preprocessor=None, smooth_idf=True,
        stop_words=None, strip_accents=None, sublinear_tf=False,
        token_pattern='(?u)\\b\\w\\w+\\b', tokenizer=None, use_idf=True,
        vocabulary=None)

In [10]:
def qstn_vectorization_tfidf(dataX, dataY):
    q_1 = []
    q_2 = []
    Y = []
    for ind in tqdm_notebook(range(len(dataX))):
        q_1.append(vectorizer.transform([get_string(dataX[i][0])]).toarray()[0])
        q_2.append(vectorizer.transform([get_string(dataX[i][1])]).toarray()[0])
        Y.append(trainY[i])
    q_1 = np.array(q_1)
    q_2 = np.array(q_2)
    X = np.concatenate((q_1, q_2), axis = 1)
    Y = np.array(Y)
    return X, Y

In [ ]:
trainX, trainY = qstn_vectorization(trainX, trainY)
testX, testY = qstn_vectorization(testX, testY)
data = {}
data['trainX'] = trainX
data['trainY'] = trainY
data['testX'] = testX
data['testY'] = testY
with open(path+'dataset_tfidf.pkl', 'wb') as fp:
    pickle.dump(data, fp)

### Glove Embedding
We are considering glove 50d data file, it will return an embedding of 50 dimension.

In [48]:
def loadGloveModel(gloveFile):
    print ("Loading Glove Model")
    f = open(gloveFile,'r', encoding='utf-8')
    model = {}
    for line in tqdm_notebook(f):
        try:
            splitLine = line.split()
            word = splitLine[0]
            embedding = np.array([float(val) for val in splitLine[1:]])
            model[word] = embedding
        except:
            print(line)
    print ("Done.",len(model)," words loaded!")
    return model

glove_model = loadGloveModel(path+'glove.6B.50d.txt')



maxlen = 0
for i in range(len(trainX)):
    maxlen = max(maxlen, max(len(trainX[i][0]), len(trainX[i][1])))
    
for i in range(len(testX)):
    maxlen = max(maxlen, max(len(testX[i][0]), len(testX[i][1])))

Loading Glove Model



Done. 400001  words loaded!


In [59]:
def get_sentence_embedding(tokens):
    zeroes = [0]*50
    vec = []
    for token in tokens:
        if token in glove_model:
            vec += list(glove_model[token])
        else:
            vec += zeroes
    for _ in range(maxlen-len(tokens)):
        vec += zeroes
    return vec

def qstn_vectorization_glove(dataX, dataY):
    q_1 = []
    q_2 = []
    Y = []
    for ind in tqdm_notebook(range(len(dataX))):
        q_1.append(get_sentence_embedding(dataX[ind][0]))
        q_2.append(get_sentence_embedding(dataX[ind][1]))
        Y.append(dataY[i])
    q_1 = np.array(q_1)
    q_2 = np.array(q_2)
    X = np.concatenate((q_1, q_2), axis = 1)
    Y = np.array(Y)
    return X, Y

In [ ]:
data = {}
trainX, trainY = qstn_vectorization_glove(trainX, trainY)
testX, testY = qstn_vectorization_glove(testX, testY)
data['trainX'] = trainX
data['trainY'] = trainY
data['testX'] = testX
data['testY'] = testY

In [ ]:
with open(path + 'dataset_glove.pkl', 'wb') as fp:
    pickle.dump(data, fp)